<a href="https://colab.research.google.com/github/JIng-Yu-Huang/generative-AI_hw/blob/main/0415Demo07aAI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_Reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Reflection 的任務設計

#### 🌟 任務說明：今日運勢占卜

**🎯 流程說明：**
1. 使用者輸入自己的星座和MBTI
2. `model_writer` 生成第一版運勢文（風格可愛調皮、看起來能說服人、使用各種emoji，使用第二人稱）
3. `model_reviewer` 檢查內容是否夠生活化、看起來可信、符合台灣文化，並提供具體修改建議
4. `model_writer` 根據建議產出第二版
5. Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 基本的設定

In [4]:
import aisuite as ai

In [5]:
provider_writer = "groq"
model_writer="llama3-70b-8192"

provider_reviewer = "groq"
model_reviewer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

標準回應函式

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hello",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「作者」和「審查員」

In [7]:
system_writer = "你是一位活潑、有趣的運勢占卜小高手，擅長依據星座和MBTI幫人占卜今日運勢、以第二人稱風格、有emoji，風格可愛風趣，但獎的畫看起來很值得信賴。請用台灣習慣的中文回應。"
system_reviewer = "你是一位潤稿專家，擅長讓運勢占卜的內容看起來更可信，用詞通順、符合台灣的文化，請針對以下星座運勢的內容給出具體修改建議。請用台灣習慣的中文回應。"

In [8]:
def Fortune_telling_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的運勢占卜：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更可信，用詞通順、符合台灣的文化。請用台灣習慣的中文, 並且只要輸出改好的運勢占卜文字就可以了。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00


In [10]:
import gradio as gr

In [11]:
with gr.Blocks() as demo:
    gr.Markdown("###🔮今日運勢占卜生成小幫手（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入你的星座和MBTI...")
    btn = gr.Button("🧝‍♀️生成運勢占卜 & 修正原始文字建議")

    with gr.Row():
        out1 = gr.Textbox(label="📜 第一版貼文 (model_writer)")
        out2 = gr.Textbox(label="📝 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label="👏 第二版貼文 (model_writer 改寫)")

    btn.click(Fortune_telling_post, inputs=[user_input], outputs=[out1, out2, out3])

In [12]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://78258e78af97d791c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://78258e78af97d791c3.gradio.live
